In [ ]:
import numpy as np

y = np.loadtxt('data/wine_data.txt', dtype=int)

x = np.arange(1, len(y) + 1)

def find_c_star(x, y, rate):
  #Generate a random probability from a normal distribution of our model generating y = +1 as a label
  c = np.random.rand()
  c_prev = c + 1
  while not np.isclose(c, c_prev):
    c_prev = c
    c += rate * np.mean(y / (y * c - ((y - 1) / 2)))
  return float(c)


find_c_star(x, y, .01)


0.7972824141696345

In [ ]:
import numpy as np

tosses = np.loadtxt('data/bayes_tosses.txt', dtype=int)

def find_posterior_distribution(t):
  num_heads = np.count_nonzero(t)
  likelihoods = []
  probability_generating_y = 0
  for k in range(0, 101):
    theta = .01 * k
    probability_generating_y += (theta ** num_heads) * ((1 - theta) ** (len(t) - num_heads))
  for k in range(0, 101):
    theta = .01 * k
    likelihood = ((theta ** num_heads) * ((1-theta) ** (len(t) - num_heads))) / probability_generating_y
    likelihoods.append(likelihood)
  return likelihoods

posterior_distribution = find_posterior_distribution(tosses)
print(f"Posterior Distribution: {posterior_distribution}")
print(f"Max likelihood of heads probability {np.count_nonzero(tosses) / len(tosses)}")
print(f"Maximum a posteriori estimate: {np.argmax(posterior_distribution) * .01}")


Posterior Distribution: [0.0, 5.316029252223897e-78, 2.9713139621418206e-62, 3.9484916743910903e-53, 1.015455062763894e-46, 8.49479483556155e-42, 8.123788449593828e-38, 1.7363126076924741e-34, 1.237457840752838e-31, 3.8065510220018783e-29, 6.027415902635356e-27, 5.570264805689705e-25, 3.2962857086926764e-23, 1.3400955384232744e-21, 3.95317750451714e-20, 8.836076241235773e-19, 1.549596909763671e-17, 2.1937967597146792e-16, 2.5670069734922892e-15, 2.5320364527344614e-14, 2.140683071958844e-13, 1.5733637051601132e-12, 1.0176106089246151e-11, 5.852920268891885e-11, 3.0211192203317e-10, 1.4106947659171033e-09, 6.000832240332104e-09, 2.3398452853370644e-08, 8.408842330981289e-08, 2.7987847212354987e-07, 8.66498064115605e-07, 2.505014556250673e-06, 6.785798573574681e-06, 1.7277772705546562e-05, 4.1464915975819825e-05, 9.403019326564316e-05, 0.0002019410145726853, 0.0004115574008639306, 0.0007973945133367461, 0.001471159070816259, 0.0025883411945722542, 0.004348326730867841, 0.0069833097271407

In [141]:
import numpy as np
from scipy.stats import norm

tosses = np.loadtxt('ps4Tosses2025.txt', dtype=int)

def find_posterior_distribution(t):
  num_heads = np.count_nonzero(t)
  likelihoods = []
  probability_generating_y = 0
  gaussian_pdf = []
  for k in range(0, 101):
    theta = .01 * k
    probability = norm.pdf(theta, loc=0.4, scale=0.2)
    gaussian_pdf.append(probability)
  sum = np.sum(gaussian_pdf)
  gaussian_pdf /= sum

  for k in range(0, 101):
    theta = .01 * k
    probability_generating_y += ((theta ** num_heads) * ((1 - theta) ** (len(t) - num_heads))) * gaussian_pdf[k]
  for k in range(0, 101):
    theta = .01 * k
    likelihood = ((theta ** num_heads) * ((1-theta) ** (len(t) - num_heads)) * gaussian_pdf[k]) / probability_generating_y
    likelihoods.append(likelihood)
  return likelihoods

posterior_distribution = find_posterior_distribution(tosses)
print(f"Posterior Distribution: {posterior_distribution}")
print(f"Max likelihood of heads probability {np.count_nonzero(tosses) / len(tosses)}")
print(f"Maximum a posteriori estimate: {np.argmax(posterior_distribution) * .01}")

Posterior Distribution: [np.float64(0.0), np.float64(9.963366205825933e-79), np.float64(6.131519570017892e-63), np.float64(8.948822817463724e-54), np.float64(2.521301390885639e-47), np.float64(2.3049450901323832e-42), np.float64(2.4028363538479677e-38), np.float64(5.584260508499164e-35), np.float64(4.316727060631661e-32), np.float64(1.436668348437092e-29), np.float64(2.4551101531986866e-27), np.float64(2.442557985038336e-25), np.float64(1.5521631555223783e-23), np.float64(6.759367037801777e-22), np.float64(2.130534607186676e-20), np.float64(5.075607206961585e-19), np.float64(9.463413777062185e-18), np.float64(1.4208239835654938e-16), np.float64(1.7587333614149607e-15), np.float64(1.8305695564376785e-14), np.float64(1.6290193712879962e-13), np.float64(1.2571145628274958e-12), np.float64(8.515565145077418e-12), np.float64(5.116875395369831e-11), np.float64(2.752420251151371e-10), np.float64(1.3360073482433337e-09), np.float64(5.892918741623682e-09), np.float64(2.376640795630806e-08), np.

In [100]:
from sklearn.datasets import fetch_openml
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


D = fetch_openml("wine", version=1, as_frame=False)
X = D.data
y = D.target

classes = ['1', '2', '3']

kf = KFold(n_splits= 4)
kf_accuracies = []
fold_num = 1
print("KFold cross validation\n")
#For each train, test split in the KFold cv, train model and report out of sample error
for train, test in kf.split(X):
  print(f"Fold {fold_num}:")
  clf = GaussianNB()
  clf.fit(X[train], y[train])
  out_of_sample_accuracy = clf.score(X[test], y[test])
  print(f"Out of Sample Accuracy: {out_of_sample_accuracy:.4f}")
  print("Class Distribution")
  for cl in classes:
    count = np.count_nonzero(y[train] == cl)
    print(f"Train Class {cl}: {count / len(y[train]):.4f}")
  for cl in classes:
    count = np.count_nonzero(y[test] == cl)
    print(f"Test Class {cl}: {count / len(y[test]):.4f}")
  kf_accuracies.append(out_of_sample_accuracy)
  fold_num += 1

mean_kf_accuracy = np.mean(kf_accuracies)
sd_kf_accuracy = np.std(kf_accuracies)
print(f"\nMean k-fold cross validation accuracy: {mean_kf_accuracy:.4f}")
print(f"Standard Deviation k-fold cross validation accuracy: {sd_kf_accuracy:.4f}")



skf = StratifiedKFold(n_splits=4)
skf_accuracies = []
fold_num = 1
print("\nStratified cross validation\n")
for train, test in skf.split(X, y):
  print(f"Fold {fold_num}:")
  clf = GaussianNB()
  clf.fit(X[train], y[train])
  out_of_sample_accuracy = clf.score(X[test], y[test])
  print(f"Out of Sample Accuracy: {out_of_sample_accuracy:.4f}")
  print("Class Distribution")
  for cl in classes:
    count = np.count_nonzero(y[train] == cl)
    print(f"Train Class {cl}: {count / len(y[train]):.4f}")
  for cl in classes:
    count = np.count_nonzero(y[test] == cl)
    print(f"Test Class {cl}: {count / len(y[test]):.4f}")
  skf_accuracies.append(out_of_sample_accuracy)
  fold_num += 1

mean_skf_accuracy = np.mean(skf_accuracies)
sd_skf_accuracy = np.std(skf_accuracies)
print(f"\nMean stratified cross validation accuracy: {mean_skf_accuracy:.4f}")
print(f"Standard Deviation stratified cross validation accuracy: {sd_skf_accuracy:.4f}")



KFold cross validation

Fold 1:
Out of Sample Accuracy: 0.8222
Class Distribution
Train Class 1: 0.1053
Train Class 2: 0.5338
Train Class 3: 0.3609
Test Class 1: 1.0000
Test Class 2: 0.0000
Test Class 3: 0.0000
Fold 2:
Out of Sample Accuracy: 0.8444
Class Distribution
Train Class 1: 0.3383
Train Class 2: 0.3008
Train Class 3: 0.3609
Test Class 1: 0.3111
Test Class 2: 0.6889
Test Class 3: 0.0000
Fold 3:
Out of Sample Accuracy: 1.0000
Class Distribution
Train Class 1: 0.4403
Train Class 2: 0.2313
Train Class 3: 0.3284
Test Class 1: 0.0000
Test Class 2: 0.9091
Test Class 3: 0.0909
Fold 4:
Out of Sample Accuracy: 0.0000
Class Distribution
Train Class 1: 0.4403
Train Class 2: 0.5299
Train Class 3: 0.0299
Test Class 1: 0.0000
Test Class 2: 0.0000
Test Class 3: 1.0000

Mean k-fold cross validation accuracy: 0.6667
Standard Deviation k-fold cross validation accuracy: 0.3909

Stratified cross validation

Fold 1:
Out of Sample Accuracy: 0.9111
Class Distribution
Train Class 1: 0.3308
Train Class

In [153]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score


D = fetch_openml("pendigits", version=1, as_frame=False)
X = D.data
y = D.target

for p in range(0, 5):
  k = 2 ** p
  clf = SVC(kernel="poly", degree = k)
  clf.fit(X, y)
  print(f"Degree {k}")
  print(f"In-sample error: {1 - clf.score(X, y):.4f}")
  print(f"Out-of-sample error: {1 - np.mean(cross_val_score(clf, X, y)):.4f}")

Degree 1
In-sample error: 0.0207
Out-of-sample error: 0.0265
Degree 2
In-sample error: 0.0043
Out-of-sample error: 0.0068
Degree 4
In-sample error: 0.0005
Out-of-sample error: 0.0055
Degree 8
In-sample error: 0.0000
Out-of-sample error: 0.0080
Degree 16
In-sample error: 0.0000
Out-of-sample error: 0.0116


In [ ]:
#Shows how important the kernel trick is by comparing it with
#regular feature expansion using PolynomialFeatures
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

D = fetch_openml("pendigits", version=1, as_frame=False)
X = D.data
y = D.target

for p in range(0, 5):
  k = 2 ** p
  poly = PolynomialFeatures(degree=k)
  X_poly = poly.fit_transform(X)
  clf = SVC(kernel="poly", degree = 1)
  clf.fit(X_poly, y)
  print(f"Degree {k}")
  print(f"In-sample error: {1 - clf.score(X_poly, y):.4f}")
  print(f"Out-of-sample error: {1 - np.mean(cross_val_score(clf, X_poly, y)):.4f}")

Degree 1
In-sample error: 0.0217
Out-of-sample error: 0.0267
Degree 2
In-sample error: 0.0083
Out-of-sample error: 0.0130
Degree 4
In-sample error: 0.0069
Out-of-sample error: 0.0096
